In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [12]:
# read dataset

df_train = pd.read_csv("/Users/fatmaoztel/Desktop/avazu-ctr-prediction/train", sep=",")
df_test = pd.read_csv("/Users/fatmaoztel/Desktop/avazu-ctr-prediction/test", sep=",")

In [20]:
seed_value = 42  # seed for reproducibility
random.seed(seed_value)

n = 40428967  # total number of records in the clickstream data 
sample_size = 1000000
skip_values = sorted(random.sample(range(1,n), n-sample_size)) # sort to skip with more efficient way
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')

In [22]:
# indicate the datatypes to use in the reading process

dtype={'id': np.dtype(int),
    'click': np.dtype(int),
    'hour': np.dtype(str),
    'C1': np.dtype(str),
    'banner_pos': np.dtype(str),
    'site_id': np.dtype(str),
    'site_domain': np.dtype(str), 
    'site_category': np.dtype(str),
    'app_id': np.dtype(str),
    'app_domain': np.dtype(str),
    'app_category': np.dtype(str),
    'device_id': np.dtype(str),
    'device_ip': np.dtype(str),
    'device_model': np.dtype(str),
    'device_type': np.dtype(str),
    'device_conn_type': np.dtype(str),
    'C14': np.dtype(str),
    'C15': np.dtype(str),
    'C16': np.dtype(str),
    'C17': np.dtype(str),
    'C18': np.dtype(str),
    'C19': np.dtype(str),
    'C20': np.dtype(str),
    'C21':np.dtype(str)
      }#

In [23]:
train = pd.read_csv("/Users/fatmaoztel/Desktop/avazu-ctr-prediction/train", parse_dates=['hour'], date_parser=parse_date, dtype=dtype, skiprows=skip_values)
test = pd.read_csv("/Users/fatmaoztel/Desktop/avazu-ctr-prediction/test")
submission = pd.read_csv('/Users/fatmaoztel/Desktop/avazu-ctr-prediction/sampleSubmission')
print('Train dataset:',train.shape)
print('Test dataset:',test.shape)
print('Submission:',submission.shape)

/var/folders/j9/scnjt0d57q3_lzqgdfj4j43r0000gn/T/ipykernel_18899/2325265658.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')


Train dataset: (1000000, 24)
Test dataset: (4577464, 23)
Submission: (4577464, 2)


In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 24 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   id                1000000 non-null  uint64        
 1   click             1000000 non-null  int64         
 2   hour              1000000 non-null  datetime64[ns]
 3   C1                1000000 non-null  object        
 4   banner_pos        1000000 non-null  object        
 5   site_id           1000000 non-null  object        
 6   site_domain       1000000 non-null  object        
 7   site_category     1000000 non-null  object        
 8   app_id            1000000 non-null  object        
 9   app_domain        1000000 non-null  object        
 10  app_category      1000000 non-null  object        
 11  device_id         1000000 non-null  object        
 12  device_ip         1000000 non-null  object        
 13  device_model      1000000 non-null  object 